In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn import datasets

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder 

from xgboost import XGBRegressor

from sklearn.metrics import accuracy_score


pd.options.mode.chained_assignment = None  # default='warn'



data_df = pd.read_csv('results.csv')
cols_to_keep = ['raceId', 'driverId', 'constructorId', 'grid','positionOrder']
df = data_df[cols_to_keep]

#On récupère les colonnes qui nous intéressent dans note banque de données



In [2]:
#Nous allons créer plus de données pour que le model soit plus précis

In [3]:
#On choisit les colonnes raceId, driverId et grid pour créer une nouvelle colonne moy_grid_by_race
moy_grid_race = data_df[["raceId","driverId","grid"]]

In [4]:
# On calcule la moyenne de grid par driver et race

def calculate_mean_grid_race(line):
    raceID = line["raceId"]
    driveID = line["driverId"]
    mean_grid = moy_grid_race.loc[(df["driverId"] == driveID) & (df["raceId"] == raceID), "grid"].mean()
    return mean_grid


In [5]:
#  et on l'ajoute à notre array principal

df["moy_grid_by_race"] = df.apply(calculate_mean_grid_race, axis = 1)


In [6]:
# on repete l'opération d'avant mais cette fois avec le constructor à la place du driver

moy_grid_constructor = data_df[["raceId","constructorId","grid"]]

In [7]:
def calculate_mean_grid_constructor(line):
    constructorID = line["constructorId"]
    driveID = line["driverId"]
    mean_grid = moy_grid_constructor.loc[(df["driverId"] == driveID) & (df["constructorId"] == constructorID), "grid"].mean()
    return mean_grid

In [8]:
df["moy_grid_by_constructor"] = df.apply(calculate_mean_grid_constructor, axis = 1)


In [9]:
# count_race_times = data_df[["raceId", "driverId"]].value_counts()


In [10]:
def racer_number_times_raced(line):
    raceID = line["raceId"]
    driveID = line["driverId"]
    if(count_race_times.loc[raceID][driveID]==3):
        print(line["driverId"])
    return count_race_times.loc[raceID][driveID]
    


In [11]:
# df["number_times_raced"] = df.apply(racer_number_times_raced, axis = 1)
#df["driverId"].value_counts().head(100)

In [12]:
# on vectorise 

vectorizer = CountVectorizer()
constructor_ids = df['constructorId'].astype(str).values.tolist()
vectorized_constructor_ids = vectorizer.fit_transform(constructor_ids)
constructor_id_vectors = vectorized_constructor_ids.toarray()
constructor_id_df = pd.DataFrame(constructor_id_vectors)
constructor_id_df = constructor_id_df.add_prefix('constructor_')
df = pd.concat([df, constructor_id_df], axis=1)
df

,raceId,driverId,constructorId,grid,positionOrder,moy_grid_by_race,moy_grid_by_constructor,constructor_0,constructor_1,constructor_2,...,constructor_191,constructor_192,constructor_193,constructor_194,constructor_195,constructor_196,constructor_197,constructor_198,constructor_199,constructor_200
0,18,1,1,1,1,1.0,4.745455,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18,2,2,5,2,5.0,9.157143,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18,3,3,7,3,7.0,10.871429,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18,4,4,11,4,11.0,6.500000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18,5,1,3,5,3.0,7.600000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,854,210,12,16,12.0,15.340909,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25836,1096,825,210,16,17,16.0,12.702970,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25837,1096,1,131,5,18,5.0,3.410000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25838,1096,849,3,20,19,20.0,16.672131,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
vectorizer = CountVectorizer()
driver_ids = df['driverId'].astype(str).values.tolist()
vectorized_driver_ids = vectorizer.fit_transform(driver_ids)
driver_id_vectors = vectorized_driver_ids.toarray()
driver_id_df = pd.DataFrame(driver_id_vectors)
driver_id_df = driver_id_df.add_prefix('driver_')
df = pd.concat([df, driver_id_df], axis=1)
df

,raceId,driverId,constructorId,grid,positionOrder,moy_grid_by_race,moy_grid_by_constructor,constructor_0,constructor_1,constructor_2,...,driver_836,driver_837,driver_838,driver_839,driver_840,driver_841,driver_842,driver_843,driver_844,driver_845
0,18,1,1,1,1,1.0,4.745455,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18,2,2,5,2,5.0,9.157143,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18,3,3,7,3,7.0,10.871429,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18,4,4,11,4,11.0,6.500000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18,5,1,3,5,3.0,7.600000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,1096,854,210,12,16,12.0,15.340909,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25836,1096,825,210,16,17,16.0,12.702970,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25837,1096,1,131,5,18,5.0,3.410000,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25838,1096,849,3,20,19,20.0,16.672131,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#X_test["positionOrder"].value_counts()

In [19]:
#drop


X = df.drop(["driverId", "constructorId"], axis = 1)

Y = df["positionOrder"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=5)

model = XGBRegressor() 

model.fit(X_train, y_train)

y_pred = model.predict(X_test) 

predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))
model.save_model('xgb_model.model')

Accuracy: 100.00%


In [16]:
model.predict(X_test.head(1)) 

array([22.999992], dtype=float32)

In [17]:
t = df[["driverId","moy_grid_by_race"]]
t.loc[2]


driverId            3.0
moy_grid_by_race    7.0
Name: 2, dtype: float64

In [18]:
count_race_times.loc[792][501]

NameError: name 'count_race_times' is not defined

In [ ]:
X = df[["grid", "moyT_grid_by_race"]]
y = df["positionOrder"]

model.fit(X, y)

model.predict(X_test)